## Dataset

In this homework, we will continue the New York City Airbnb Open Data. You can take it from Kaggle or download from here if you don't want to sign up to Kaggle.

We'll keep working with the 'price' variable, and we'll transform it to a classification task.

Features

For the rest of the homework, you'll need to use the features from the previous homework with additional two 'neighbourhood_group' and 'room_type'. So the whole feature set will be set as follows:

'neighbourhood_group',
'room_type',
'latitude',
'longitude',
'price',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'availability_365'
Select only them and fill in the missing values with 0.

In [127]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import wget

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

from IPython.display import display

In [2]:
##data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv'

In [3]:
##wget.download(data, 'aribnbny.csv')

In [4]:
df = pd.read_csv('AB_NYC_2019.csv')
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


In [5]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [6]:
df_subset = df[['neighbourhood_group', 'room_type', 'latitude', 'longitude',
              'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month',
              'calculated_host_listings_count', 'availability_365']]

In [7]:
df_subset

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Private room,40.64749,-73.97237,149,1,9,0.21,6,365
1,Manhattan,Entire home/apt,40.75362,-73.98377,225,1,45,0.38,2,355
2,Manhattan,Private room,40.80902,-73.94190,150,3,0,NaN,1,365
3,Brooklyn,Entire home/apt,40.68514,-73.95976,89,1,270,4.64,1,194
4,Manhattan,Entire home/apt,40.79851,-73.94399,80,10,9,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...
48890,Brooklyn,Private room,40.67853,-73.94995,70,2,0,NaN,2,9
48891,Brooklyn,Private room,40.70184,-73.93317,40,4,0,NaN,2,36
48892,Manhattan,Entire home/apt,40.81475,-73.94867,115,10,0,NaN,1,27
48893,Manhattan,Shared room,40.75751,-73.99112,55,1,0,NaN,6,2


In [8]:
df_subset.isnull().sum()

neighbourhood_group                   0
room_type                             0
latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [9]:
df_subset['reviews_per_month'] = df_subset.reviews_per_month.fillna(0)

<ipython-input-9-ce4e2398c591>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['reviews_per_month'] = df_subset.reviews_per_month.fillna(0)


In [10]:
df_subset.isnull().sum()

neighbourhood_group               0
room_type                         0
latitude                          0
longitude                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

## Question 1

What is the most frequent observation (mode) for the column 'neighbourhood_group'?

In [11]:
df_subset.neighbourhood_group.describe()

count         48895
unique            5
top       Manhattan
freq          21661
Name: neighbourhood_group, dtype: object

In [12]:
df_subset.neighbourhood_group.mode()

0    Manhattan
dtype: object

## Split the data

Split your data in train/val/test sets, with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value ('price') is not in your dataframe.

In [13]:
df_full_train, df_test = train_test_split(df_subset, test_size = 0.2, random_state = 42)

In [14]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)

In [15]:
len(df_subset), len(df_train), len(df_val), len(df_test)

(48895, 29337, 9779, 9779)

In [16]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [17]:
y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

In [18]:
del df_train['price']
del df_val['price']
del df_test['price']

In [19]:
df_train.dtypes

neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

## Question 2

Create the correlation matrix for the numerical features of your train dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
What are the two features that have the biggest correlation in this dataset?
Example of a correlation matrix for the car price dataset:

In [20]:
df_train.corrwith(df_train).abs()

latitude                          1.0
longitude                         1.0
minimum_nights                    1.0
number_of_reviews                 1.0
reviews_per_month                 1.0
calculated_host_listings_count    1.0
availability_365                  1.0
dtype: float64

In [21]:
df_train

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Entire home/apt,40.72760,-73.94495,3,29,0.70,13,50
1,Manhattan,Private room,40.70847,-74.00498,1,0,0.00,1,7
2,Bronx,Entire home/apt,40.83149,-73.92766,40,0,0.00,1,0
3,Brooklyn,Entire home/apt,40.66448,-73.99407,2,3,0.08,1,0
4,Manhattan,Private room,40.74118,-74.00012,1,48,1.80,2,67
...,...,...,...,...,...,...,...,...,...
29332,Brooklyn,Private room,40.71748,-73.95685,6,5,0.13,1,0
29333,Brooklyn,Private room,40.66397,-73.98538,1,7,0.17,2,0
29334,Manhattan,Private room,40.79994,-73.97001,1,1,0.64,1,88
29335,Brooklyn,Private room,40.69585,-73.96344,60,0,0.00,1,0


In [22]:
df_train.corr(method='pearson', min_periods=1)

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


reviews_per_month, number_of_reviews, corr 59.0374%

## Make price binary

We need to turn the price variable from numeric into binary.
Let's create a variable above_average which is 1 if the price is above (or equal to) 152.

In [23]:
dv = DictVectorizer(sparse=False)

In [24]:
above_average = df['price'].values >= 152

In [25]:
above_average = above_average.astype(int)

In [26]:
above_average

array([0, 1, 0, ..., 0, 0, 0])

## Question 3

Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.
Which of these two variables has bigger score?
Round it to 2 decimal digits using round(score, 2)

In [27]:
df_train.dtypes

neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [28]:
y_train

array([ 99,  57,  70, ..., 299,  65,  92])

In [29]:
above_average_train = y_train >= 152

In [30]:
above_average_train = above_average_train.astype(int)

In [33]:
round(mutual_info_score(df_train.neighbourhood_group, above_average_train), 2)

0.05

In [34]:
round(mutual_info_score(df_train.room_type, above_average_train), 2)

0.14

## Question 4

Now let's train a logistic regression
Remember that we have two categorical variables in the data. Include them using one-hot encoding.
Fit the model on the training dataset.
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.

In [36]:
df_train[['neighbourhood_group', 'room_type']]

,neighbourhood_group,room_type
0,Brooklyn,Entire home/apt
1,Manhattan,Private room
2,Bronx,Entire home/apt
3,Brooklyn,Entire home/apt
4,Manhattan,Private room
...,...,...
29332,Brooklyn,Private room
29333,Brooklyn,Private room
29334,Manhattan,Private room
29335,Brooklyn,Private room


In [38]:
train_dicts = df_train.to_dict(orient = 'records')

In [39]:
train_dicts[0]

{'neighbourhood_group': 'Brooklyn',
 'room_type': 'Entire home/apt',
 'latitude': 40.7276,
 'longitude': -73.94495,
 'minimum_nights': 3,
 'number_of_reviews': 29,
 'reviews_per_month': 0.7,
 'calculated_host_listings_count': 13,
 'availability_365': 50}

In [40]:
dv = DictVectorizer(sparse=False)

In [42]:
X_train = dv.fit_transform(train_dicts)

In [44]:
X_train.shape

(29337, 15)

In [43]:
dv.get_feature_names()

['availability_365',
 'calculated_host_listings_count',
 'latitude',
 'longitude',
 'minimum_nights',
 'neighbourhood_group=Bronx',
 'neighbourhood_group=Brooklyn',
 'neighbourhood_group=Manhattan',
 'neighbourhood_group=Queens',
 'neighbourhood_group=Staten Island',
 'number_of_reviews',
 'reviews_per_month',
 'room_type=Entire home/apt',
 'room_type=Private room',
 'room_type=Shared room']

In [45]:
X_train

array([[ 50.     ,  13.     ,  40.7276 , ...,   1.     ,   0.     ,
          0.     ],
       [  7.     ,   1.     ,  40.70847, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.83149, ...,   1.     ,   0.     ,
          0.     ],
       ...,
       [ 88.     ,   1.     ,  40.79994, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.69585, ...,   0.     ,   1.     ,
          0.     ],
       [281.     ,   2.     ,  40.64438, ...,   1.     ,   0.     ,
          0.     ]])

In [62]:
val_dicts = df_val.to_dict(orient='records')

In [64]:
X_val = dv.transform(val_dicts)

In [65]:
X_val

array([[ 52.     ,   1.     ,  40.70239, ...,   0.     ,   1.     ,
          0.     ],
       [343.     ,   2.     ,  40.68498, ...,   1.     ,   0.     ,
          0.     ],
       [260.     ,   1.     ,  40.66911, ...,   1.     ,   0.     ,
          0.     ],
       ...,
       [  0.     ,   1.     ,  40.73058, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.68858, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.81882, ...,   1.     ,   0.     ,
          0.     ]])

In [72]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, above_average_train)

/Users/tonygunawan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [73]:
model.intercept_

array([-0.00252852])

In [74]:
model.coef_[0].round(3)

array([ 0.003,  0.004, -0.217, -0.089, -0.012, -0.378,  0.084,  1.198,
       -0.783, -0.124, -0.003, -0.037,  1.637, -1.19 , -0.449])

In [75]:
y_pred = model.predict_proba(X_train)[:, 1]

In [76]:
y_pred = model.predict_proba(X_val)[:, 1]

In [77]:
churn_decision = (y_pred >= 0.5)

In [78]:
df_val[churn_decision]

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
1,Brooklyn,Entire home/apt,40.68498,-73.96618,14,4,0.11,2,343
4,Manhattan,Entire home/apt,40.76075,-73.99893,30,0,0.00,18,365
6,Manhattan,Entire home/apt,40.73243,-74.00932,4,1,0.03,1,0
7,Manhattan,Entire home/apt,40.80630,-73.96268,7,2,0.05,1,0
9,Manhattan,Entire home/apt,40.80861,-73.94574,3,123,1.73,3,248
...,...,...,...,...,...,...,...,...,...
9763,Manhattan,Entire home/apt,40.73525,-73.98074,14,0,0.00,1,0
9768,Manhattan,Entire home/apt,40.80177,-73.95710,8,9,0.18,1,111
9770,Manhattan,Entire home/apt,40.73833,-73.98943,2,2,0.70,1,166
9774,Manhattan,Entire home/apt,40.75974,-73.98924,4,2,0.14,1,5


In [79]:
above_average_val = y_val >= 152
above_average_test = y_test >= 152

In [80]:
above_average_val = above_average_val.astype(int)
above_average_test = above_average_test.astype(int)

In [82]:
above_average_test

array([0, 0, 0, ..., 0, 1, 0])

In [85]:
round((above_average_val == churn_decision).mean(), 2)

0.79

## Question 5

We have 9 features: 7 numerical features and 2 categorical.
Let's find the least useful one using the feature elimination technique.
Train a model with all these features (using the same parameters as in Q4).
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
Which of following feature has the smallest difference?
neighbourhood_group
room_type
number_of_reviews
reviews_per_month
note: the difference doesn't have to be positive

In [86]:
df_train.dtypes

neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [87]:
train_dicts = df_train.to_dict(orient = 'records')

In [88]:
train_dicts[0]

{'neighbourhood_group': 'Brooklyn',
 'room_type': 'Entire home/apt',
 'latitude': 40.7276,
 'longitude': -73.94495,
 'minimum_nights': 3,
 'number_of_reviews': 29,
 'reviews_per_month': 0.7,
 'calculated_host_listings_count': 13,
 'availability_365': 50}

In [89]:
dv = DictVectorizer(sparse=False)

In [90]:
X_train = dv.fit_transform(train_dicts)

In [91]:
X_train.shape

(29337, 15)

In [92]:
dv.get_feature_names()

['availability_365',
 'calculated_host_listings_count',
 'latitude',
 'longitude',
 'minimum_nights',
 'neighbourhood_group=Bronx',
 'neighbourhood_group=Brooklyn',
 'neighbourhood_group=Manhattan',
 'neighbourhood_group=Queens',
 'neighbourhood_group=Staten Island',
 'number_of_reviews',
 'reviews_per_month',
 'room_type=Entire home/apt',
 'room_type=Private room',
 'room_type=Shared room']

In [93]:
X_train

array([[ 50.     ,  13.     ,  40.7276 , ...,   1.     ,   0.     ,
          0.     ],
       [  7.     ,   1.     ,  40.70847, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.83149, ...,   1.     ,   0.     ,
          0.     ],
       ...,
       [ 88.     ,   1.     ,  40.79994, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.69585, ...,   0.     ,   1.     ,
          0.     ],
       [281.     ,   2.     ,  40.64438, ...,   1.     ,   0.     ,
          0.     ]])

In [94]:
val_dicts = df_val.to_dict(orient='records')

In [95]:
X_val = dv.transform(val_dicts)

In [96]:
X_val

array([[ 52.     ,   1.     ,  40.70239, ...,   0.     ,   1.     ,
          0.     ],
       [343.     ,   2.     ,  40.68498, ...,   1.     ,   0.     ,
          0.     ],
       [260.     ,   1.     ,  40.66911, ...,   1.     ,   0.     ,
          0.     ],
       ...,
       [  0.     ,   1.     ,  40.73058, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.68858, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.81882, ...,   1.     ,   0.     ,
          0.     ]])

In [97]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, above_average_train)

/Users/tonygunawan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [98]:
model.intercept_

array([-0.00252852])

In [99]:
model.coef_[0].round(3)

array([ 0.003,  0.004, -0.217, -0.089, -0.012, -0.378,  0.084,  1.198,
       -0.783, -0.124, -0.003, -0.037,  1.637, -1.19 , -0.449])

In [100]:
y_pred = model.predict_proba(X_train)[:, 1]

In [101]:
y_pred = model.predict_proba(X_val)[:, 1]

In [102]:
churn_decision = (y_pred >= 0.5)

In [122]:
acc = (above_average_val == churn_decision).mean()

In [103]:
round((above_average_val == churn_decision).mean(), 2)

0.79

In [105]:
df_train_mng = df_train.copy()

In [107]:
del df_train_mng['neighbourhood_group']

In [118]:
train_dicts_mng = df_train_mng.to_dict(orient = 'records')
X_train_mng = dv.fit_transform(train_dicts_mng)

df_val_mng = df_val.copy()
val_dicts_mng = df_val_mng.to_dict(orient='records')
X_val_mng = dv.transform(val_dicts_mng)

model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train_mng, above_average_train)

y_pred_mng = model.predict_proba(X_train_mng)[:, 1]
y_pred_mng = model.predict_proba(X_val_mng)[:, 1]

churn_decision_mng = (y_pred_mng >= 0.5)
##round((above_average_val == churn_decision_mng).mean(), 2)
acc_mng = (above_average_val == churn_decision_mng).mean()
acc_mng

0.7509970344616014

In [114]:
train_dicts_mng[0]

{'room_type': 'Entire home/apt',
 'latitude': 40.7276,
 'longitude': -73.94495,
 'minimum_nights': 3,
 'number_of_reviews': 29,
 'reviews_per_month': 0.7,
 'calculated_host_listings_count': 13,
 'availability_365': 50}

In [119]:
df_train_mrt = df_train.copy()
del df_train_mrt['room_type']

train_dicts_mrt = df_train_mrt.to_dict(orient = 'records')
X_train_mrt = dv.fit_transform(train_dicts_mrt)

df_val_mrt = df_val.copy()
val_dicts_mrt = df_val_mrt.to_dict(orient='records')
X_val_mrt = dv.transform(val_dicts_mrt)

model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train_mrt, above_average_train)

y_pred_mrt = model.predict_proba(X_train_mrt)[:, 1]
y_pred_mrt = model.predict_proba(X_val_mrt)[:, 1]

churn_decision_mrt = (y_pred_mrt >= 0.5)
##acc_mrt = round((above_average_val == churn_decision_mrt).mean(), 2)
acc_mrt = (above_average_val == churn_decision_mrt).mean()
acc_mrt

/Users/tonygunawan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7166376930156457

In [113]:
train_dicts_mrt[0]

{'neighbourhood_group': 'Brooklyn',
 'latitude': 40.7276,
 'longitude': -73.94495,
 'minimum_nights': 3,
 'number_of_reviews': 29,
 'reviews_per_month': 0.7,
 'calculated_host_listings_count': 13,
 'availability_365': 50}

In [120]:
df_train_mnor = df_train.copy()
del df_train_mnor['number_of_reviews']

train_dicts_mnor = df_train_mnor.to_dict(orient = 'records')
X_train_mnor = dv.fit_transform(train_dicts_mnor)

df_val_mnor = df_val.copy()
val_dicts_mnor = df_val_mnor.to_dict(orient='records')
X_val_mnor = dv.transform(val_dicts_mnor)

model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train_mnor, above_average_train)

y_pred_mnor = model.predict_proba(X_train_mnor)[:, 1]
y_pred_mnor = model.predict_proba(X_val_mnor)[:, 1]

churn_decision_mnor = (y_pred_mnor >= 0.5)
##acc_mnor = round((above_average_val == churn_decision_mnor).mean(), 2)
acc_mnor = (above_average_val == churn_decision_mnor).mean()
acc_mnor

/Users/tonygunawan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7868902750792515

In [121]:
df_train_mrpm = df_train.copy()
del df_train_mrpm['reviews_per_month']

train_dicts_mrpm = df_train_mrpm.to_dict(orient = 'records')
X_train_mrpm = dv.fit_transform(train_dicts_mrpm)

df_val_mrpm = df_val.copy()
val_dicts_mrpm = df_val_mrpm.to_dict(orient='records')
X_val_mrpm = dv.transform(val_dicts_mrpm)

model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train_mrpm, above_average_train)

y_pred_mrpm = model.predict_proba(X_train_mrpm)[:, 1]
y_pred_mrpm = model.predict_proba(X_val_mrpm)[:, 1]

churn_decision_mrpm = (y_pred_mrpm >= 0.5)
##acc_mrpm = round((above_average_val == churn_decision_mrpm).mean(), 2)
acc_mrpm = (above_average_val == churn_decision_mrpm).mean()
acc_mrpm

/Users/tonygunawan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.785356375907557

In [123]:
acc - acc_mnor

-0.0006135596686778211

In [124]:
acc - acc_mrpm

0.0009203395030166206

## Question 6

For this question, we'll see how to use a linear regression model from Scikit-Learn
We'll need to use the original column 'price'. Apply the logarithmic transformation to this column.
Fit the Ridge regression model on the training data.
This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]
Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.
If there are multiple options, select the smallest alpha.

In [125]:
y_train

array([ 99,  57,  70, ..., 299,  65,  92])

In [145]:
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)
y_test_log = np.log1p(y_test)

In [146]:
y_train_log

array([4.60517019, 4.06044301, 4.26267988, ..., 5.70378247, 4.18965474,
       4.53259949])

In [147]:
df['price'].describe()

count    48895.000000
mean       152.720687
std        240.154170
min          0.000000
25%         69.000000
50%        106.000000
75%        175.000000
max      10000.000000
Name: price, dtype: float64

In [144]:
np.log(1)

0.0

In [148]:
model = Ridge(alpha = 0)
model.fit(X_train, y_train_log)


Ridge(alpha=0)

In [163]:
model.predict(X_val)

array([4.09765625, 5.21484375, 5.0078125 , ..., 4.5625    , 4.18359375,
       5.15234375])

In [164]:
y_pred = model.predict(X_val)

In [159]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [170]:
round(rmse(y_val, y_pred), 3)

268.916

In [171]:
model = Ridge(alpha = 0.01)
model.fit(X_train, y_train_log)

y_pred = model.predict(X_val)

round(rmse(y_val, y_pred), 3)


268.914

In [172]:
model = Ridge(alpha = 0.1)
model.fit(X_train, y_train_log)

y_pred = model.predict(X_val)

round(rmse(y_val, y_pred), 3)



268.914

In [173]:
model = Ridge(alpha = 1)
model.fit(X_train, y_train_log)

y_pred = model.predict(X_val)

round(rmse(y_val, y_pred), 3)



268.914

In [174]:
model = Ridge(alpha = 10)
model.fit(X_train, y_train_log)

y_pred = model.predict(X_val)

round(rmse(y_val, y_pred), 3)



268.916

0.01 with 268.914 